In [1]:
import torch
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

from requests import get
import re

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [24]:
# Ler um arquivo em texto puro contendo o conteúdo a ser aprendido
file_len = len(file)
print('file_len =', file_len)

symbols = " \,\.\!\?\*\(\)\"\:\…\―\@\$\%"
file = open('lfv_texts_plain.txt', encoding='utf-8').read()
words = set(re.findall("[^\n"+symbols+"]+", file))
symbols = symbols.split('\\')
symbols.append('\n')
all_words = symbols + list(words)
n_words = len(all_words)


print(all_words, n_words)

file_len = 1298370
[' ', ',', '.', '!', '?', '*', '(', ')"', ':', '…', '―', '@', '$', '%', '\n', 'visual', '72', 'Nenhuma', 'postas', 'Ssssssssônia', 'Hiih', 'setores', 'borracha', 'Engraçado', 'prorrogações', 'partida', 'consultoria', 'Gardel', 'compartilhou', 'reúnem', 'rasgar', 'sofisticou', 'inatacável', 'exóticos', 'resisti-lo', 'Turquia', 'desunir', 'montar', 'atenderem', 'bonito', 'tossindo', 'persistia', 'autores', 'não', 'parágrafo', 'Leivas', 'rumou', 'errada', 'prove', 'criança', 'ex', 'existiam', 'Caixeiro', 'fregueses', 'fazíamos', 'Perdi', 'Randal', 'Pércicles', 'indignação', 'integraram', 'concorrentes', 'Quero', 'Desvendei', 'Lancha', 'imagino', 'justificando', 'Tudo', 'ortodoxo', 'Giovanna', 'percorrer', 'Morreu', 'contaram', 'treinados', 'kitch', 'Jó', 'batendo', 'renúncia', 'eterizado', 'Capitão', '~//~', 'Faça', 'dramático', 'inquisição', 'vizinha', 'tricolor', 'alegorias', 'hotel-fazenda', 'participar', 'leitão', 'pssts', 'Opa', 'Brigadeiro', 'suja',

In [25]:
chunk_len = 30

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

i que eu sou eu. O que você gr


In [28]:
# Converte palavras para uma lista de inteiros

def word_tensor(words):
    tensor = torch.zeros(len(words)).long()
    for w in range(len(words)):
      try:
        tensor[w] = all_words.index(words[w])
      except:
        print(w)
        raise
    return Variable(tensor)

print(word_tensor(['?', 'Caixeiro', 'Valdemira']))

tensor([  4,  52, 128])


In [34]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk = random_chunk()
    inp = word_tensor(chunk[:-1])
    target = word_tensor(chunk[1:])
    return inp, target

In [30]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.lstm(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        size = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        return (size, size)

In [36]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden()
    hidden = [h.cuda() for h in hidden]
    prime_input = word_tensor(prime_str).cuda()
    predicted = prime_str

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_word = all_words[top_i]
        predicted += predicted_word
        inp = word_tensor(predicted_word).cuda()

    return predicted

In [37]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = [h.cuda() for h in hidden]
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [38]:
n_epochs = 50000
print_every = 10
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_words, hidden_size, n_words, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Os', 100), '\n')


RuntimeError: cuda runtime error (38) : no CUDA-capable device is detected at /opt/conda/conda-bld/pytorch_1549635019666/work/aten/src/THC/THCGeneral.cpp:51

In [0]:
# Com a rede treinada, gerar um texto longo

print(evaluate('O psicanalista de', 5500), '\n')